In [1]:
import os
import shutil
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import (
    EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB3,
    EfficientNetB4, EfficientNetB5, EfficientNetB6, EfficientNetB7,
    EfficientNetV2B0, EfficientNetV2B1, EfficientNetV2B2, EfficientNetV2B3,
    EfficientNetV2S, EfficientNetV2M, EfficientNetV2L
)
from tensorflow.keras.applications.efficientnet import preprocess_input as effnetv1_preprocess
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input as effnetv2_preprocess
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# Set seeds for reproducibility
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Training source directories
train_sources = [
    '/kaggle/input/chestxray-augment/train',
    '/kaggle/input/chest-xray-pneumonia/chest_xray/train',
    '/kaggle/input/dcgan-pneumonia/chest_xray/train'
]

# Test directory
test_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/test'

# Combined training directory
combined_train_dir = '/kaggle/working/combined_train'

# Image and training parameters
BATCH_SIZE = 32
EPOCHS = 64
IMG_SIZE = (224, 224)

# Combine all training sources into one directory
if os.path.exists(combined_train_dir):
    shutil.rmtree(combined_train_dir)

os.makedirs(os.path.join(combined_train_dir, 'NORMAL'))
os.makedirs(os.path.join(combined_train_dir, 'PNEUMONIA'))

def copy_images(src_dir, dest_dir):
    for label in ['NORMAL', 'PNEUMONIA']:
        src_label_dir = os.path.join(src_dir, label)
        dest_label_dir = os.path.join(dest_dir, label)
        if os.path.exists(src_label_dir):
            for fname in os.listdir(src_label_dir):
                src_path = os.path.join(src_label_dir, fname)
                dest_path = os.path.join(dest_label_dir, fname)
                if os.path.isfile(src_path):
                    shutil.copy(src_path, dest_path)

# Copy from all sources
for src in train_sources:
    copy_images(src, combined_train_dir)

# Data Generators
def get_generators(preprocess_func):
    train_gen = ImageDataGenerator(
        preprocessing_function=preprocess_func,
        horizontal_flip=True,
        zoom_range=0.2,
        rotation_range=15
    ).flow_from_directory(
        combined_train_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        seed=SEED
    )

    test_gen = ImageDataGenerator(
        preprocessing_function=preprocess_func
    ).flow_from_directory(
        test_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False
    )
    return train_gen, test_gen

# Build classifier head
def build_model(base_model):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=output)
    return model

# EfficientNet Models Dictionary (Only)
models = {
    'EfficientNetB0': (EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv1_preprocess),
    'EfficientNetB1': (EfficientNetB1(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv1_preprocess),
    'EfficientNetB2': (EfficientNetB2(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv1_preprocess),
    'EfficientNetB3': (EfficientNetB3(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv1_preprocess),
    'EfficientNetB4': (EfficientNetB4(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv1_preprocess),
    'EfficientNetB5': (EfficientNetB5(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv1_preprocess),
    'EfficientNetB6': (EfficientNetB6(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv1_preprocess),
    'EfficientNetB7': (EfficientNetB7(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv1_preprocess),
    'EfficientNetV2B0': (EfficientNetV2B0(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv2_preprocess),
    'EfficientNetV2B1': (EfficientNetV2B1(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv2_preprocess),
    'EfficientNetV2B2': (EfficientNetV2B2(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv2_preprocess),
    'EfficientNetV2B3': (EfficientNetV2B3(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv2_preprocess),
    'EfficientNetV2S': (EfficientNetV2S(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv2_preprocess),
    'EfficientNetV2M': (EfficientNetV2M(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv2_preprocess),
    'EfficientNetV2L': (EfficientNetV2L(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv2_preprocess),
}

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.2)

# Result storage
report_rows = []

# Training and Evaluation Loop
for model_name, (base_model, preprocess_func) in models.items():
    print(f"\n🔧 Training model: {model_name}")
    train_gen, test_gen = get_generators(preprocess_func)
    model = build_model(base_model)

    for layer in base_model.layers:
        layer.trainable = False

    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(
        train_gen,
        epochs=EPOCHS,
        validation_data=test_gen,
        callbacks=[early_stop, reduce_lr],
        verbose=2
    )

    predictions = model.predict(test_gen, verbose=0)
    predicted_labels = (predictions > 0.5).astype(int).ravel()
    true_labels = test_gen.classes

    accuracy = accuracy_score(true_labels, predicted_labels)
    report = classification_report(true_labels, predicted_labels, digits=6, output_dict=True)

    row = {
        'Model': model_name,
        'Accuracy': accuracy,
        'Macro Precision': report['macro avg']['precision'],
        'Macro Recall': report['macro avg']['recall'],
        'Macro F1': report['macro avg']['f1-score'],
        'Weighted Precision': report['weighted avg']['precision'],
        'Weighted Recall': report['weighted avg']['recall'],
        'Weighted F1': report['weighted avg']['f1-score'],
    }

    report_rows.append(row)

# Save to CSV
df_report = pd.DataFrame(report_rows)
df_report.to_csv('/kaggle/working/classification_summary.csv', index=False)
print("✅ Saved summary to /kaggle/working/classification_summary.csv")
print("📊 Classification Summary:")
print(df_report)


2025-07-21 10:44:08.137864: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753094648.329007      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753094648.380486      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0000 00:00:1753094793.614897      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
27018416/27018416 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
31790344/31790344 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
43941136/43941136 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
71686520/71686520 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
115263384/115263384 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
165234480/165234480 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step
258076736/258076736 ━━━━━━━━━━━━━━━━━━━━ 9s 0us/step
24274472/24274472 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
28456008/28456008 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
35839040/35839040 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
52606240/52606240 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
82420632/82420632 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
214201816/214201816 ━━━━━━━━━━━━━━━━━━━━ 8s 0us/step
473176280/473176280 ━━━━━━━━━━━━━━━━━━━━ 16s 0us/step

🔧 Training model: EfficientNetB0
Found 21224 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/64


I0000 00:00:1753094925.399165      57 service.cc:148] XLA service 0x7eabc4005980 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753094925.400100      57 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753094927.435082      57 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753094935.948304      57 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

Epoch 1/64
664/664 - 335s - 504ms/step - accuracy: 0.8852 - loss: 0.2773 - val_accuracy: 0.8558 - val_loss: 0.3391 - learning_rate: 0.0010
Epoch 2/64
664/664 - 240s - 362ms/step - accuracy: 0.9012 - loss: 0.2399 - val_accuracy: 0.8942 - val_loss: 0.2433 - learning_rate: 0.0010
Epoch 3/64
664/664 - 243s - 365ms/step - accuracy: 0.9061 - loss: 0.2308 - val_accuracy: 0.8317 - val_loss: 0.4648 - learning_rate: 0.0010
Epoch 4/64
664/664 - 242s - 364ms/step - accuracy: 0.9089 - loss: 0.2211 - val_accuracy: 0.8734 - val_loss: 0.3530 - learning_rate: 0.0010
Epoch 5/64
664/664 - 242s - 365ms/step - accuracy: 0.9106 - loss: 0.2146 - val_accuracy: 0.8990 - val_loss: 0.2562 - learning_rate: 2.0000e-04

🔧 Training model: EfficientNetV2M
Found 21224 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/64
664/664 - 376s - 566ms/step - accuracy: 0.8789 - loss: 0.2871 - val_accuracy: 0.9022 - val_loss: 0.2527 - learning_rate: 0.0010
Epoch 2/64
664/664 - 249s - 375ms/step - accuracy: 0.8965 - loss: 0.2504 - val_accuracy: 0.8942 - val_loss: 0.2404 - learning_rate: 0.0010
Epoch 3/64
664/664 - 248s - 374ms/step - accuracy: 0.9024 - loss: 0.2380 - val_accuracy: 0.9119 - val_loss: 0.2175 - learning_rate: 0.0010
Epoch 4/64
664/664 - 248s - 373ms/step - accuracy: 0.9041 - loss: 0.2338 - val_accuracy: 0.9022 - val_loss: 0.2364 - learning_rate: 0.0010
Epoch 5/64
664/664 - 246s - 371ms/step - accuracy: 0.9057 - loss: 0.2292 - val_accuracy: 0.8958 - val_loss: 0.2238 - learning_rate: 0.0010
Epoch 6/64
664/664 - 246s - 371ms/step - accuracy: 0.9113 - loss: 0.2178 - val_accuracy: 0.9135 - val_loss: 0.2196 - learning_rate: 2.0000e-04

🔧 Training model: EfficientNetV2L
Found 21224 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/64
664/664 - 440s - 663ms/step - accuracy: 0.8810 - loss: 0.2845 - val_accuracy: 0.8702 - val_loss: 0.3242 - learning_rate: 0.0010
Epoch 2/64
664/664 - 254s - 382ms/step - accuracy: 0.8998 - loss: 0.2428 - val_accuracy: 0.8862 - val_loss: 0.3004 - learning_rate: 0.0010
Epoch 3/64
664/664 - 254s - 383ms/step - accuracy: 0.9007 - loss: 0.2394 - val_accuracy: 0.8862 - val_loss: 0.3022 - learning_rate: 0.0010
Epoch 4/64
664/664 - 251s - 378ms/step - accuracy: 0.9020 - loss: 0.2359 - val_accuracy: 0.8878 - val_loss: 0.3071 - learning_rate: 0.0010
Epoch 5/64
664/664 - 254s - 382ms/step - accuracy: 0.9108 - loss: 0.2162 - val_accuracy: 0.8846 - val_loss: 0.3051 - learning_rate: 2.0000e-04
✅ Saved summary to /kaggle/working/classification_summary.csv
📊 Classification Summary:
               Model  Accuracy  Macro Precision  Macro Recall  Macro F1  \
0     EfficientNetB0  0.908654         0.908463      0.895299  0.901048   
1     EfficientNetB1  0.924679         0.932703      0.907265  